mobilenet_v3_large with softmax, Adam

In [1]:
import torch
import torchvision
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import matplotlib.pyplot as plt
from torchinfo import summary
import os
from sklearn.model_selection import train_test_split
from PIL import Image
import torch.optim as optim
from torchvision.models import swin_s
from torchvision.models import mobilenet_v3_large
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from collections import OrderedDict
from tqdm import tqdm

In [2]:
from elpv_reader import load_dataset
images, proba, types = load_dataset()

In [3]:
# check version number
import imblearn
from imblearn.over_sampling import SMOTE

train_set, test_set, train_lables, test_lables = \
train_test_split(images, proba, test_size=0.25,random_state=42)

my_mapping = {0.0:0, 0.3333333333333333:1, 0.6666666666666666:2, 1.0:3}

for i in range(len(train_set)):
    train_lables[i] = my_mapping[train_lables[i]]

for i in range(len(test_set)):
    test_lables[i] = my_mapping[test_lables[i]]

In [4]:
count_list = [0,0,0,0]
for i in range(len(train_set)):
    if train_lables[i] == 0:
        count_list[0] += 1
    if train_lables[i] == 1:
        count_list[1] += 1
    if train_lables[i] == 2:
        count_list[2] += 1
    if train_lables[i] == 3:
        count_list[3] += 1 
for i, j in zip(['The 0.0 has:', 'The 0.33 has:', 'The 0.66 has:', 'The 1.0 has:'], count_list):
    print(i, j)

The 0.0 has: 1136
The 0.33 has: 222
The 0.66 has: 75
The 1.0 has: 535


In [5]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=False):
        self.images = images
        self.label_mapping = {0: np.eye(4)[0], 1: np.eye(4)[1],\
                              2: np.eye(4)[2], 3: np.eye(4)[3]}
        self.labels = [self.label_mapping[label] for label in labels]
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        image = Image.fromarray(image)

        # 如果图像为灰度图像，将其复制到三个通道，转换为 RGB 图像
        if image.mode == 'L':
            image = image.convert('RGB')

        if self.transform:
            image = self.transform(image)
        return image, label


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
    ])


custom_train_dataset = CustomDataset(train_set, train_lables, transform)
custom_test_dataset = CustomDataset(test_set, test_lables, transform)


batch_size = 32
train_loader = DataLoader(custom_train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(custom_test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")  
else:
    device = torch.device("cpu")  

model = mobilenet_v3_large(weights=True)

classifier1 = nn.Sequential(OrderedDict([('fc1', nn.Linear(960, 4)),
                           ('output', nn.Softmax())
                          ]))
    
model.classifier = classifier1

model.to(device)
print()

d:\anaconda\envs\torchtest\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:

lossfunc = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

save_path = '.\model.pth'
best_accuracy = 79
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)  # 移动数据到 CUDA
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = lossfunc(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs} - Training Loss: {train_loss:.4f}')


    model.eval()
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)  
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            _, real_labels = torch.max(labels, 1)
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(real_labels.cpu().numpy())

    
    accuracy = accuracy_score(all_labels, all_predictions)
    conf_matrix = confusion_matrix(all_labels, all_predictions)

    if best_accuracy < accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), save_path)
        print(f'Epoch [{epoch + 1}/{num_epochs}] - Saved Best Model (Best Accuracy: {best_accuracy:.4f})')

    print(f'Accuracy: {accuracy:.4f}')
    print('Confusion Matrix:')
    print(conf_matrix)

  0%|          | 0/62 [00:00<?, ?it/s]d:\anaconda\envs\torchtest\lib\site-packages\torch\nn\modules\container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
100%|██████████| 62/62 [00:08<00:00,  6.89it/s]


Epoch 1/20 - Training Loss: 1.4357
Accuracy: 0.2180
Confusion Matrix:
[[ 95   0 276   1]
 [ 17   0  56   0]
 [  9   0  22   0]
 [ 72   0  82  26]]


  0%|          | 0/62 [00:00<?, ?it/s]d:\anaconda\envs\torchtest\lib\site-packages\torch\nn\modules\container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
100%|██████████| 62/62 [00:06<00:00, 10.26it/s]


Epoch 2/20 - Training Loss: 1.3652
Accuracy: 0.2912
Confusion Matrix:
[[138   0 114 120]
 [ 23   0  29  21]
 [ 16   0   2  13]
 [ 93   5  31  51]]


  0%|          | 0/62 [00:00<?, ?it/s]d:\anaconda\envs\torchtest\lib\site-packages\torch\nn\modules\container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
100%|██████████| 62/62 [00:06<00:00,  9.87it/s]


...

Epoch 18/20 - Training Loss: 0.9721
Accuracy: 0.7363
Confusion Matrix:
[[346   4   0  22]
 [ 59   8   0   6]
 [ 25   0   0   6]
 [ 51   0   0 129]]


  0%|          | 0/62 [00:00<?, ?it/s]d:\anaconda\envs\torchtest\lib\site-packages\torch\nn\modules\container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
100%|██████████| 62/62 [00:06<00:00,  9.75it/s]


Epoch 19/20 - Training Loss: 0.9672
Accuracy: 0.7363
Confusion Matrix:
[[349   3   0  20]
 [ 59   8   0   6]
 [ 25   0   0   6]
 [ 54   0   0 126]]


  0%|          | 0/62 [00:00<?, ?it/s]d:\anaconda\envs\torchtest\lib\site-packages\torch\nn\modules\container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
100%|██████████| 62/62 [00:06<00:00,  9.81it/s]


Epoch 20/20 - Training Loss: 0.9615
Accuracy: 0.7409
Confusion Matrix:
[[351   1   0  20]
 [ 58   8   0   7]
 [ 23   0   0   8]
 [ 53   0   0 127]]


In [8]:
# 创建两个子集，分别存储mono和poly的标签和预测
mono_subset_labels = [label for label, img_type in zip(all_labels, types) if img_type == "mono"]
mono_subset_predictions = [pred for pred, img_type in zip(all_predictions, types) if img_type == "mono"]
poly_subset_labels = [label for label, img_type in zip(all_labels, types) if img_type == "poly"]
poly_subset_predictions = [pred for pred, img_type in zip(all_predictions, types) if img_type == "poly"]

# 计算不同类型的准确率和混淆矩阵
accuracy = accuracy_score(all_labels, all_predictions)
accuracy_mono = accuracy_score(mono_subset_labels, mono_subset_predictions)
accuracy_poly = accuracy_score(poly_subset_labels, poly_subset_predictions)
conf_matrix = confusion_matrix(all_labels, all_predictions)
conf_matrix_mono = confusion_matrix(mono_subset_labels, mono_subset_predictions)
conf_matrix_poly = confusion_matrix(poly_subset_labels, poly_subset_predictions)
f1_total = f1_score(all_labels, all_predictions, average='macro')
f1_mono = f1_score(mono_subset_labels, mono_subset_predictions, average='macro')
f1_poly = f1_score(poly_subset_labels, poly_subset_predictions, average='macro')

print('The total accuracy is:', accuracy)
print()
print('The accuracy of mono type is:', accuracy_mono)
print()
print('The accuracy of poly type is:', accuracy_poly)
print()
print('The total confusion matrix is:\n', conf_matrix)
print()
print('The confusion matrix of mono type is:\n', conf_matrix_mono)
print()
print('The confusion matrix of poly type is:\n', conf_matrix_poly)
print()
print('The total F1 Score is:', f1_total)
print()
print('The F1 Score of mono type is:', f1_mono)
print()
print('The F1 Score of poly type is:', f1_poly)


The total accuracy is: 0.7408536585365854

The accuracy of mono type is: 0.7

The accuracy of poly type is: 0.7644230769230769

The total confusion matrix is:
 [[351   1   0  20]
 [ 58   8   0   7]
 [ 23   0   0   8]
 [ 53   0   0 127]]

The confusion matrix of mono type is:
 [[125   0   0   9]
 [ 25   2   0   2]
 [  9   0   0   3]
 [ 24   0   0  41]]

The confusion matrix of poly type is:
 [[226   1   0  11]
 [ 33   6   0   5]
 [ 14   0   0   5]
 [ 29   0   0  86]]

The total F1 Score is: 0.4392371331132092

The F1 Score of mono type is: 0.40025228113021943

The F1 Score of poly type is: 0.4617764823647177
